In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets
import statsmodels.api as sm
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import auc,roc_auc_score,confusion_matrix

In [2]:
data = pd.read_csv("OOT_Sample.csv")
A = data.iloc[:,1:282]
C=pd.DataFrame(A)
c=C.fillna(C.mean())
X=pd.DataFrame(c)

In [3]:
#data["y"] = data["y"].astype('bool')
y = data["y"] 

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(234959, 281) (234959,)
(58740, 281) (58740,)


In [5]:
model = LogisticRegression(random_state=123)
model.fit(X_train, y_train)

C:\Users\IN16\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\IN16\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=123, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [6]:
y_prob=model.predict_proba(X_train)

In [7]:
ks = pd.DataFrame()
ks['score'] = y_prob[:,1]
ks['actual'] = data['y'].reset_index(drop = True)
ks = ks.sort_values(by = 'score' , ascending = False).reset_index(drop=True)
ks['decile'] = pd.cut(ks.index,10, labels= range(10))
ks['bad'] = ks['actual']
ks['goods'] = 1 - ks['bad']

agg1 = pd.DataFrame()
agg1['min_scr'] = ks.groupby(by = 'decile').score.min()
agg1['max_scr'] = ks.groupby(by = 'decile').score.max()
agg1['bad'] = ks.groupby(by = 'decile').bad.sum()
agg1['goods'] = ks.groupby(by = 'decile').goods.sum()
agg1['total'] = agg1['bad'] + agg1['goods']
agg1['bad_rate'] = agg1['bad']/(agg1['bad'].sum())
agg1['cumbad'] = agg1['bad_rate'].cumsum()
agg1['good_rate'] = agg1['goods']/(agg1['goods'].sum())
agg1['cumgoods'] = agg1['good_rate'].cumsum()
agg1["ks"] = agg1['cumgoods'] - agg1['cumbad']
agg1

,min_scr,max_scr,bad,goods,total,bad_rate,cumbad,good_rate,cumgoods,ks
decile,,,,,,,,,,
0,0.193526,1.000000,1507,21989,23496,0.102107,0.102107,0.099859,0.099859,-0.002248
1,0.156901,0.193526,1481,22015,23496,0.100346,0.202453,0.099977,0.199837,-0.002616
2,0.129205,0.156898,1454,22042,23496,0.098516,0.300969,0.100100,0.299936,-0.001032
3,0.107372,0.129205,1501,21995,23496,0.101701,0.402670,0.099886,0.399823,-0.002847
4,0.083988,0.107372,1479,22017,23496,0.100210,0.502880,0.099986,0.499809,-0.003070
5,0.073857,0.083987,1493,22002,23495,0.101159,0.604038,0.099918,0.599728,-0.004311
6,0.065154,0.073857,1422,22074,23496,0.096348,0.700386,0.100245,0.699973,-0.000413
7,0.052942,0.065154,1466,22030,23496,0.099329,0.799715,0.100045,0.800018,0.000303
8,0.022788,0.052942,1482,22014,23496,0.100413,0.900129,0.099973,0.899991,-0.000138


In [8]:
ks_score=max(agg1["ks"])
ks_score*100

0.030273742975162055

In [9]:
model = RandomForestClassifier(random_state=123)
#fit function
model.fit(X_train, y_train)

C:\Users\IN16\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=123, verbose=0, warm_start=False)

In [10]:
y_pred=model.predict(X_train)

In [11]:
y_prob=model.predict_proba(X_train)

In [12]:
ks = pd.DataFrame()
ks['score'] = y_prob[:,1]
ks['actual'] = data['y'].reset_index(drop = True)
ks = ks.sort_values(by = 'score' , ascending = False).reset_index(drop=True)
ks['decile'] = pd.cut(ks.index,10, labels= range(10))
ks['bad'] = ks['actual']
ks['goods'] = 1 - ks['bad']

agg1 = pd.DataFrame()
agg1['min_scr'] = ks.groupby(by = 'decile').score.min()
agg1['max_scr'] = ks.groupby(by = 'decile').score.max()
agg1['bad'] = ks.groupby(by = 'decile').bad.sum()
agg1['goods'] = ks.groupby(by = 'decile').goods.sum()
agg1['total'] = agg1['bad'] + agg1['goods']
agg1['bad_rate'] = agg1['bad']/(agg1['bad'].sum())
agg1['cumbad'] = agg1['bad_rate'].cumsum()
agg1['good_rate'] = agg1['goods']/(agg1['goods'].sum())
agg1['cumgoods'] = agg1['good_rate'].cumsum()
agg1["ks"] = agg1['cumgoods'] - agg1['cumbad']
agg1

,min_scr,max_scr,bad,goods,total,bad_rate,cumbad,good_rate,cumgoods,ks
decile,,,,,,,,,,
0,0.1,1.0,1422,22074,23496,0.096348,0.096348,0.100245,0.100245,3.897241e-03
1,0.1,0.1,1432,22064,23496,0.097026,0.193374,0.100200,0.200445,7.071515e-03
2,0.0,0.1,1512,21984,23496,0.102446,0.295819,0.099837,0.300282,4.462062e-03
3,0.0,0.0,1521,21975,23496,0.103056,0.398875,0.099796,0.400077,1.201940e-03
4,0.0,0.0,1417,22079,23496,0.096009,0.494884,0.100268,0.500345,5.460664e-03
5,0.0,0.0,1440,22055,23495,0.097568,0.592452,0.100159,0.600504,8.052024e-03
6,0.0,0.0,1490,22006,23496,0.100955,0.693407,0.099936,0.700441,7.033096e-03
7,0.0,0.0,1514,21982,23496,0.102581,0.795989,0.099827,0.800268,4.279050e-03
8,0.0,0.0,1450,22046,23496,0.098245,0.894234,0.100118,0.900386,6.151986e-03


In [14]:
ks_score=max(agg1["ks"])
ks_score*100

0.805202404555061

In [15]:
X_train=X
y_train=y
y_train1=y

In [16]:
sel = SelectFromModel(RandomForestClassifier(random_state=123))
sel.fit(X_train, y_train)

C:\Users\IN16\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=123, verbose=0, warm_start=False),
        max_features=None, norm_order=1, prefit=False, threshold=None)

In [17]:
sel.get_support()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False, False, False, False,  True,  True,  True, False,
        True,  True, False, False, False, False, False,  True,  True,
       False, False, False, False,  True,  True, False,  True,  True,
        True, False, False, False,  True,  True,  True, False, False,
       False, False,  True, False, False, False, False, False,  True,
        True,  True, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False, False,  True,  True, False,  True, False,
        True,  True, False, False,  True,  True, False, False, False,
       False, False, False, False, False, False, False, False,  True,
        True, False,  True,  True, False, False, False,  True, False,
       False, False,

In [18]:
selected_feat= X_train.columns[(sel.get_support())]
len(selected_feat)

65

In [19]:
print(selected_feat)

Index(['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x15',
       'x16', 'x17', 'x19', 'x20', 'x26', 'x27', 'x32', 'x33', 'x35', 'x36',
       'x37', 'x41', 'x42', 'x43', 'x48', 'x54', 'x55', 'x56', 'x61', 'x72',
       'x82', 'x84', 'x85', 'x86', 'x87', 'x88', 'x89', 'x90', 'x91', 'x92',
       'x95', 'x96', 'x98', 'x100', 'x101', 'x104', 'x105', 'x117', 'x118',
       'x120', 'x121', 'x125', 'x147', 'x148', 'x149', 'x150', 'x167', 'x168',
       'x170', 'x172', 'x175', 'x176', 'x177', 'x184'],
      dtype='object')


In [20]:
X[selected_feat]

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x149,x150,x167,x168,x170,x172,x175,x176,x177,x184
0,0.0000,0.0000,0.009145,0.017202,0.0000,-0.0420,0.002436,0.0,0.0,0.0,...,1032.922959,3.647800,0.002000,0.43530,0.129400,0.000000,-0.339600,-0.415300,0.071900,0.561474
1,0.0000,0.0000,0.009145,0.017202,0.0000,-0.0445,0.002436,0.0,0.0,0.0,...,0.350000,0.891800,-0.291800,0.85010,-0.248300,0.000000,0.882400,1.504100,0.185000,0.561474
2,0.0134,-0.0229,0.009145,0.017202,0.0000,-0.0229,0.002436,0.0,1.0,0.0,...,-8869.000000,22.763100,-0.160900,0.36500,-0.892200,0.000000,0.829800,15.978600,-0.304600,0.561474
3,0.0000,0.0000,0.000000,0.000000,0.0000,0.0000,0.000000,0.0,0.0,0.0,...,0.032700,2.151000,0.271000,0.50630,-0.004300,0.000000,-0.939200,-0.938900,0.010400,0.561474
4,-0.0007,0.0183,0.009145,0.017202,-0.0007,0.0000,0.002436,1.0,0.0,0.0,...,0.195700,0.968700,-0.019100,-620.71871,-0.024500,0.000000,-0.411500,-0.396800,0.099400,0.561474
5,0.0320,0.0320,0.009145,0.017202,0.0000,0.0000,0.002436,1.0,1.0,0.0,...,1032.922959,439.376757,1870.214366,-620.71871,-24086.854271,1.000000,81.919889,108.103133,19.443946,0.561474
6,0.0091,0.0030,0.009145,0.017202,0.0000,0.0192,0.002436,1.0,1.0,1.0,...,1032.922959,439.376757,1870.214366,-620.71871,-24086.854271,0.000000,81.919889,108.103133,19.443946,0.561474
7,0.0000,0.0000,0.009145,0.017202,0.0000,0.0038,0.002436,0.0,0.0,1.0,...,1032.922959,-1.639200,1870.214366,-620.71871,-24086.854271,1.000000,81.919889,108.103133,19.443946,0.561474
8,-0.0076,-0.0076,0.009145,0.017202,0.0000,-0.0887,0.002436,0.0,0.0,0.0,...,1032.922959,439.376757,1870.214366,-620.71871,-24086.854271,0.000000,81.919889,108.103133,19.443946,0.561474
9,0.0000,0.0000,0.000000,0.000000,0.0000,0.0083,0.000000,0.0,0.0,1.0,...,-0.011200,0.015500,1870.214366,-620.71871,-24086.854271,0.000000,81.919889,108.103133,-0.011000,0.561474


In [21]:
X_train1=X[selected_feat]

In [22]:
clf = RandomForestClassifier(random_state=123)
clf.fit(X_train1, y_train1)

C:\Users\IN16\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=123, verbose=0, warm_start=False)

In [23]:
a=[]
b=[]
importances = clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X_train1.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))
    a.append(indices[f])
    b.append(importances[indices[f]])
'''
# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()
'''

Feature ranking:
1. feature 5 (0.042876)
2. feature 29 (0.038330)
3. feature 48 (0.034866)
4. feature 21 (0.033741)
5. feature 25 (0.033615)
6. feature 1 (0.030393)
7. feature 20 (0.029735)
8. feature 15 (0.028456)
9. feature 24 (0.028322)
10. feature 13 (0.025861)
11. feature 31 (0.023111)
12. feature 44 (0.023022)
13. feature 23 (0.021774)
14. feature 0 (0.021500)
15. feature 35 (0.021456)
16. feature 46 (0.021070)
17. feature 37 (0.019373)
18. feature 56 (0.019040)
19. feature 51 (0.018956)
20. feature 22 (0.018793)
21. feature 54 (0.017967)
22. feature 53 (0.017098)
23. feature 18 (0.016295)
24. feature 3 (0.016060)
25. feature 26 (0.015757)
26. feature 50 (0.015328)
27. feature 36 (0.014931)
28. feature 59 (0.014754)
29. feature 42 (0.014561)
30. feature 57 (0.014448)
31. feature 19 (0.013921)
32. feature 14 (0.013914)
33. feature 41 (0.013701)
34. feature 32 (0.013600)
35. feature 40 (0.013479)
36. feature 28 (0.013322)
37. feature 2 (0.012905)
38. feature 39 (0.012593)
39. featu

'\n# Plot the feature importances of the forest\nplt.figure()\nplt.title("Feature importances")\nplt.bar(range(X.shape[1]), importances[indices],\n       color="r", yerr=std[indices], align="center")\nplt.xticks(range(X.shape[1]), indices)\nplt.xlim([-1, X.shape[1]])\nplt.show()\n'

In [24]:
FI=pd.DataFrame()

In [25]:
FI['variable']=a

In [26]:
FI['importance']=b

In [27]:
print(FI)

    variable  importance
0          5    0.042876
1         29    0.038330
2         48    0.034866
3         21    0.033741
4         25    0.033615
5          1    0.030393
6         20    0.029735
7         15    0.028456
8         24    0.028322
9         13    0.025861
10        31    0.023111
11        44    0.023022
12        23    0.021774
13         0    0.021500
14        35    0.021456
15        46    0.021070
16        37    0.019373
17        56    0.019040
18        51    0.018956
19        22    0.018793
20        54    0.017967
21        53    0.017098
22        18    0.016295
23         3    0.016060
24        26    0.015757
25        50    0.015328
26        36    0.014931
27        59    0.014754
28        42    0.014561
29        57    0.014448
..       ...         ...
35        28    0.013322
36         2    0.012905
37        39    0.012593
38         4    0.011520
39        43    0.010933
40        38    0.010025
41        16    0.010000
42        64    0.009758


In [28]:
d=np.cumsum(b)
(d<=0.90).argmin()

48

In [29]:
H=a[0:48]

In [30]:
print(H)

[5, 29, 48, 21, 25, 1, 20, 15, 24, 13, 31, 44, 23, 0, 35, 46, 37, 56, 51, 22, 54, 53, 18, 3, 26, 50, 36, 59, 42, 57, 19, 14, 41, 32, 40, 28, 2, 39, 4, 43, 38, 16, 64, 34, 49, 63, 30, 47]


In [31]:
H.sort()

In [32]:
print(H)

[0, 1, 2, 3, 4, 5, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 53, 54, 56, 57, 59, 63, 64]


In [33]:
var=[]
for i in H:
    var.append(X.columns[i])
print(var)

['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x14', 'x15', 'x16', 'x17', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x29', 'x30', 'x31', 'x32', 'x33', 'x35', 'x36', 'x37', 'x38', 'x39', 'x40', 'x41', 'x42', 'x43', 'x44', 'x45', 'x47', 'x48', 'x49', 'x50', 'x51', 'x52', 'x54', 'x55', 'x57', 'x58', 'x60', 'x64', 'x65']


In [34]:
X[var]

,x1,x2,x3,x4,x5,x6,x14,x15,x16,x17,...,x50,x51,x52,x54,x55,x57,x58,x60,x64,x65
0,0.0000,0.0000,0.009145,0.017202,0.0000,-0.0420,0.078917,0.415294,0.000000,0.200000,...,0.000000,0.00000,0.000000,0.301900,0.0,1.000000,0.00000,0.182865,0.000000,1.000000
1,0.0000,0.0000,0.009145,0.017202,0.0000,-0.0445,0.078917,0.415294,0.088148,0.189756,...,0.000675,0.00033,0.000000,0.477700,0.0,0.000000,0.00000,1.000000,0.000000,1.000000
2,0.0134,-0.0229,0.009145,0.017202,0.0000,-0.0229,0.078917,0.415294,0.088148,0.189756,...,0.000675,0.00033,0.000000,-0.879000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000
3,0.0000,0.0000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.088148,0.189756,...,0.000675,0.00033,0.000000,-0.007100,0.0,1.000000,0.00000,0.000000,0.000000,1.000000
4,-0.0007,0.0183,0.009145,0.017202,-0.0007,0.0000,0.078917,0.415294,0.022653,0.250000,...,0.000675,0.00033,0.000000,0.194100,3.0,0.000000,0.00000,1.000000,0.000000,0.000000
5,0.0320,0.0320,0.009145,0.017202,0.0000,0.0000,0.078917,0.415294,0.088148,0.189756,...,0.000000,0.00000,0.000000,230.423889,0.0,0.000000,0.00000,0.182865,1.000000,0.000000
6,0.0091,0.0030,0.009145,0.017202,0.0000,0.0192,0.078917,0.415294,0.000000,0.000000,...,0.000000,0.00000,0.000135,230.423889,2.0,0.000000,0.00000,0.182865,0.172729,0.077395
7,0.0000,0.0000,0.009145,0.017202,0.0000,0.0038,0.078917,0.415294,0.000000,0.000000,...,0.000675,0.00033,0.000000,230.423889,2.0,0.000000,0.00000,0.000000,0.000000,0.000000
8,-0.0076,-0.0076,0.009145,0.017202,0.0000,-0.0887,0.078917,0.415294,0.000000,0.000000,...,0.000675,0.00033,0.000000,230.423889,0.0,0.000000,0.00000,0.182865,0.000000,0.000000
9,0.0000,0.0000,0.000000,0.000000,0.0000,0.0083,0.000000,0.000000,0.088148,0.189756,...,0.000675,0.00033,0.000000,230.423889,4.0,0.000000,0.00000,1.000000,0.000000,0.000000


In [35]:
X_train, X_test, y_train, y_test = train_test_split(X[var], y, test_size=0.2, random_state=123)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(234959, 48) (234959,)
(58740, 48) (58740,)


In [36]:
model = LogisticRegression(random_state=123)
#fit function
model.fit(X_train, y_train)

C:\Users\IN16\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\IN16\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=123, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [37]:
logit = sm.Logit(y_train,X_train)

In [38]:
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.227113
         Iterations 15


In [39]:
print(result.summary())

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:               234959
Model:                          Logit   Df Residuals:                   234917
Method:                           MLE   Df Model:                           41
Date:                Tue, 28 May 2019   Pseudo R-squ.:                 0.03153
Time:                        11:51:45   Log-Likelihood:                -53362.
converged:                       True   LL-Null:                       -55099.
                                        LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.5347      0.737     -0.726      0.468      -1.979       0.909
x2            -2.2500      0.572     -3.935      0.000      -3.371      -1.129
x3             0.0184      0.104      0.176      0.8

In [40]:
X1=X[var]

In [41]:
X2=X1.drop(['x1','x3','x15','x23','x24','x25','x26','x29','x30','x32','x33','x37','x38','x40','x41','x43','x44','x45','x47','x48','x49','x50','x51','x52','x54','x57'], axis=1)

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.2, random_state=123)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(234959, 22) (234959,)
(58740, 22) (58740,)


In [43]:
logit = sm.Logit(y_train,X_train)

In [44]:
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.227171
         Iterations 9


In [45]:
print(result.summary())

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:               234959
Model:                          Logit   Df Residuals:                   234937
Method:                           MLE   Df Model:                           21
Date:                Tue, 28 May 2019   Pseudo R-squ.:                 0.03128
Time:                        11:58:33   Log-Likelihood:                -53376.
converged:                       True   LL-Null:                       -55099.
                                        LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x2            -2.4266      0.434     -5.595      0.000      -3.277      -1.577
x4             0.3794      0.056      6.830      0.000       0.270       0.488
x5            -3.4284      0.858     -3.995      0.0

In [58]:
model = RandomForestClassifier(n_estimators=50, min_samples_split=2, min_samples_leaf=1, max_features='auto', max_depth=None, random_state=123)
#fit function
model.fit(X2, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=40, n_jobs=None,
            oob_score=False, random_state=123, verbose=0, warm_start=False)

In [59]:
y_prob=model.predict_proba(X2)

In [60]:
ks = pd.DataFrame()
ks['score'] = y_prob[:,1]
ks['actual'] = data['y'].reset_index(drop = True)
ks = ks.sort_values(by = 'score' , ascending = False).reset_index(drop=True)
ks['decile'] = pd.cut(ks.index,10, labels= range(10))
ks['bad'] = ks['actual']
ks['goods'] = 1 - ks['bad']

agg1 = pd.DataFrame()
agg1['min_scr'] = ks.groupby(by = 'decile').score.min()
agg1['max_scr'] = ks.groupby(by = 'decile').score.max()
agg1['bad'] = ks.groupby(by = 'decile').bad.sum()
agg1['goods'] = ks.groupby(by = 'decile').goods.sum()
agg1['total'] = agg1['bad'] + agg1['goods']
agg1['bad_rate'] = agg1['bad']/(agg1['bad'].sum())
agg1['cumbad'] = agg1['bad_rate'].cumsum()
agg1['good_rate'] = agg1['goods']/(agg1['goods'].sum())
agg1['cumgoods'] = agg1['good_rate'].cumsum()
agg1["ks"] = agg1['cumbad'] - agg1['cumgoods']
agg1

,min_scr,max_scr,bad,goods,total,bad_rate,cumbad,good_rate,cumgoods,ks
decile,,,,,,,,,,
0,0.100,1.000,18381,10989,29370,0.995181,0.995181,0.039927,0.039927,0.955255
1,0.050,0.100,76,29294,29370,0.004115,0.999296,0.106435,0.146362,0.852934
2,0.025,0.050,13,29357,29370,0.000704,1.000000,0.106664,0.253026,0.746974
3,0.025,0.025,0,29370,29370,0.000000,1.000000,0.106711,0.359737,0.640263
4,0.025,0.025,0,29370,29370,0.000000,1.000000,0.106711,0.466448,0.533552
5,0.000,0.025,0,29369,29369,0.000000,1.000000,0.106708,0.573155,0.426845
6,0.000,0.000,0,29370,29370,0.000000,1.000000,0.106711,0.679867,0.320133
7,0.000,0.000,0,29370,29370,0.000000,1.000000,0.106711,0.786578,0.213422
8,0.000,0.000,0,29370,29370,0.000000,1.000000,0.106711,0.893289,0.106711


In [61]:
ks_score=max(agg1["ks"])
ks_score*100

95.52546232982544

In [ ]:
ks = pd.DataFrame()
ks['score'] = y_prob[:,1]
ks['actual'] = data['y'].reset_index(drop = True)
ks = ks.sort_values(by = 'score' , ascending = False).reset_index(drop=True)
ks['decile'] = pd.cut(ks.index,10, labels= range(10))
ks['bad'] = ks['actual']
ks['goods'] = 1 - ks['bad']

agg1 = pd.DataFrame()
agg1['min_scr'] = ks.groupby(by = 'decile').score.min()
agg1['max_scr'] = ks.groupby(by = 'decile').score.max()
agg1['bad'] = ks.groupby(by = 'decile').bad.sum()
agg1['goods'] = ks.groupby(by = 'decile').goods.sum()
agg1['total'] = agg1['bad'] + agg1['goods']
agg1['bad_rate'] = agg1['bad']/(agg1['bad'].sum())
agg1['cumbad'] = agg1['bad_rate'].cumsum()
agg1['good_rate'] = agg1['goods']/(agg1['goods'].sum())
agg1['cumgoods'] = agg1['good_rate'].cumsum()
agg1["ks"] = agg1['cumbad'] - agg1['cumgoods']
agg1